# EVRP Algorithm
___


### Index

___


In [1]:
import sys
import os
from pathlib import Path 
sys.path.insert(1 ,os.path.dirname(Path(os.path.abspath("__file__")).resolve().parent))

import math
import folium
import pickle
import numpy as np
import pandas as pd
from geopy import Point, Nominatim
from geopy.distance import geodesic
import itertools 

import pyomo.solvers
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

from datetime import datetime

In [2]:
# initial environ settings 
file_path = "../data/input/EVRP - Template.xlsx"
model = pyo.ConcreteModel()
opt = SolverFactory('glpk')

In [3]:
# Constantes
velocidade_media = 20
tempo_serviço = 0.5

In [4]:
# Cálculo de distância entre pontos i e j
def calcular_distancia(latitude_i, longitude_i, latitude_j, longitude_j):
    return math.ceil(geodesic(Point(latitude=latitude_i, longitude=longitude_i), Point(latitude=latitude_j, longitude=longitude_j)).km)


In [5]:
# CONJUNTOS
# Clientes
df_clientes = pd.read_excel(file_path, sheet_name='Clientes')
clientes = df_clientes.to_dict("records")
clientes = {
    cliente['Cliente']: {
        'Latitude': cliente['Latitude'],
        'Longitude': cliente['Longitude']
    }
    for cliente in clientes
}

# Pedidos
df_pedidos = pd.read_excel(file_path, sheet_name='Pedidos')
pedidos = df_pedidos.to_dict("records")
pedidos = {
    pedido['Pedido']: {
        'Cliente': pedido['Cliente'],
        'Quantidade': pedido['Quantidade'],
        'Leadtime': pedido['Leadtime']
    }
    for pedido in pedidos
}

# Pontos de Recarga
df_pontos_recarga = pd.read_csv("../data/input/charging_stations.csv", sep=";", decimal=".", encoding="utf-8")
pontos_recarga = df_pontos_recarga.to_dict("records")
pontos_recarga = {
    ponto['ID']: {
        'Latitude': ponto['Latitude'],
        'Longitude': ponto['Longitude'],
        'Potência de Recarga': ponto['Potência de Recarga']
    }
    for ponto in pontos_recarga
}

# Veículos
df_veiculos = pd.read_excel(file_path, sheet_name='Veículos')
veiculos = df_veiculos.to_dict("records")
veiculos = {
    veiculo['Veículo']: {
        'Capacidade da Bateria (kWh)': veiculo['Capacidade da Bateria (kWh)'],
        'Consumo (kWh/km)': veiculo['Consumo (kWh/km)']
    }
    for veiculo in veiculos
}

# Centros de Distribuição
df = pd.read_excel(file_path, sheet_name='Centro de Distribuição')
centros_distribuicao = df.to_dict("records")
centros_distribuicao = {
    centro['Centro de Distribuição']: {
        'Latitude': centro['Latitude'],
        'Longitude': centro['Longitude']
    }
    for centro in centros_distribuicao
}


# Todos os pontos
pontos = {}
pontos.update(clientes)
pontos.update(pontos_recarga)
pontos.update(centros_distribuicao)

In [6]:
# Sets
model.C = pyo.Set(initialize=clientes.keys(), doc='Clientes')
model.R = pyo.Set(initialize=pontos_recarga.keys(), doc='Pontos de Recarga')
model.P = pyo.Set(initialize=pedidos.keys(), doc='Pedidos')
model.K = pyo.Set(initialize=veiculos.keys(), doc='Veículos')
model.zero = pyo.Set(initialize=centros_distribuicao.keys(), doc='Centros de Distribuição')

model.N = pyo.Set(initialize=model.C.union(model.R), doc='Conjunto de pontos - Clientes e Pontos de Recarga')
model.Nlinha = pyo.Set(initialize=model.N.union(model.zero), doc='Conjuntos de pontos - N e Centros de Distribuição')

In [7]:
# Parametros
def atribuir_distancia(model, i, j):
    try:
        return calcular_distancia(pontos[i]['Latitude'], pontos[i]['Longitude'], pontos[j]['Latitude'], pontos[j]['Longitude'])
    except KeyError:
        return 0
model.d = pyo.Param(model.Nlinha, model.Nlinha, initialize=atribuir_distancia, doc='Distância entre pontos i e j (km)')

def atribuir_capacidade(model, k):
    try:
        return veiculos[k]['Capacidade da Bateria (kWh)']
    except KeyError:
        return 0
model.Q = pyo.Param(model.K, within=pyo.NonNegativeIntegers, initialize=atribuir_capacidade, doc='Capacidade da bateria do veículo (kWh)')

model.v = pyo.Param(within=pyo.NonNegativeIntegers, initialize=velocidade_media, doc='Velocidade média dos Veículos (km/h)')

def atribuir_consumo(model, k):
    try:
        return veiculos[k]['Consumo (kWh/km)']
    except KeyError:
        return 0
model.c = pyo.Param(model.K, within=pyo.NonNegativeReals, initialize=atribuir_consumo, doc='Consumo por Km do veículo (kWh/km)')

def atribuir_potencia_recarga(model, r):
    try:
        return pontos_recarga[r]['Potência de Recarga']
    except KeyError:
        return 0
model.r = pyo.Param(model.R, within=pyo.NonNegativeReals, initialize=atribuir_potencia_recarga, doc='Potência de recarga do ponto de recarga (kWh)')

def atribuir_demanda(model, p):
    try:
        return pedidos[p]['Quantidade']
    except KeyError:
        return 0
model.q = pyo.Param(model.P, within=pyo.NonNegativeReals, initialize=atribuir_demanda, doc='Demanda do pedido do cliente i (unidades)')

def atribuir_leadtime(model, p):
    try:
        return pedidos[p]['Leadtime']
    except KeyError:
        return 0
model.l = pyo.Param(model.P, within=pyo.NonNegativeReals, initialize=atribuir_leadtime, doc='Leadtime do pedido do cliente i (horas)')

model.s = pyo.Param(within=pyo.NonNegativeReals, initialize=tempo_serviço, doc='Tempo de serviço (horas)')

model.tzero = pyo.Param(within=pyo.NonNegativeReals, initialize=0, doc='Tempo inicial (horas)')

In [ ]:
# Variáveis de decisão
model.x = pyo.Var(model.K, model.Nlinha, model.Nlinha, within=pyo.Binary, doc='Variável de decisão que indica se o veículo 𝑘 vai de i para j')
model.y = pyo.Var(model.K, model.Nlinha, within=pyo.NonNegativeReals, doc='Variável de decisão que indica se o veículo 𝑘 atende o cliente i')
model.t = pyo.Var(model.K, model.P, within=pyo.NonNegativeReals, doc='Variável de decisão que indica o tempo de chegada do veículo 𝑘 no ponto i')
model.u = pyo.Var(model.K, model.R, within=pyo.NonNegativeReals, doc='Variável de decisão que indica a carga adicionada ao veículo 𝑘 no ponto i')
model.z = pyo.Var(model.K, model.R, within=pyo.Binary, doc='Variável binária que indica se o veículo recarrega no ponto i')

ERROR: Rule failed when initializing variable for Var x with index ('V12',
'R532', 'C295'): MemoryError:
ERROR: Constructing component 'x' from data=None failed:
        MemoryError:
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\LEDmundo\Downloads\tcc\EVRP\env\Lib\site-packages\IPython\core\interactiveshell.py", line 2168, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=tb_offset
    )
  File "c:\Users\LEDmundo\Downloads\tcc\EVRP\env\Lib\site-packages\IPython\core\ultratb.py", line 1457, in structured_traceback
    return FormattedTB.structured_traceback(
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        self, etype, evalue, etb, tb_offset, number_of_lines_of_context
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\LEDmundo\Downloads\tcc\EVRP\env\Lib\site-packages\IPython\core\ultratb.py", line 1348, in structured_traceback
    return VerboseTB.structured_traceback(
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        self, etype, value, tb, tb_offset, number_of_lines_of_context
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    

: 

In [ ]:
# Função Objetivo
def FuncaoObj(model):
    return sum(model.x[k, i, j] * model.d[i, j] for k in model.K for i in model.Nlinha for j in model.Nlinha)
model.obj = pyo.Objective(rule=FuncaoObj, sense=pyo.minimize, doc='Função Objetivo para Otimização do Problema de Roteirização de Veículos Elétricos - EVRP')

In [ ]:
# Restrições
def atendimento_pedido_rule(model, i):
    return sum(model.x[k, i, j] for k in model.K for j in model.Nlinha) >= 1
model.atendimento_cliente = pyo.Constraint(model.C, rule=atendimento_pedido_rule, doc='Restrição de atendimento ao cliente i')

def capacidade_bateria_rule(model, k, i):
    return model.y[k, i] <= model.Q[k]/model.c[k]
model.capacidade_bateria = pyo.Constraint(model.K, model.C, rule=capacidade_bateria_rule, doc='Restrição de capacidade da bateria do veículo k')

def carregamento_veiculo_rule(model, k, i):
    return model.y[k, i] + (model.u[k, i] * model.z[k, i]) <= model.Q[k]/model.c[k]
model.carregamento_veiculo = pyo.Constraint(model.K, model.Nlinha, rule=carregamento_veiculo_rule, doc='Restrição de carregamento do veículo k')

def recarga_apenas_em_pontos_de_recarga_rule(model, k, i):
    if i not in model.R:
        return model.z[k, i] == 0
    return pyo.Constraint.Skip
model.recarga_pontos_recarga = pyo.Constraint(model.K, model.Nlinha, rule=recarga_apenas_em_pontos_de_recarga_rule, doc='Restrição de recarga apenas em pontos de recarga')

def autonomia_rule(model, k, i, j):
    return model.y[k, i] - model.y[k, j] >= model.d[i, j] * model.x[k, i, j]
model.autonomia = pyo.Constraint(model.K, model.Nlinha, model.Nlinha, rule=autonomia_rule, doc='Restrição de autonomia do veículo k')

def leadtime_rule(model, k, p):
    return model.t[k, p] + model.s[p] <= model.demand[p]
model.leadtime = pyo.Constraint(model.K, model.P, rule=leadtime_rule, doc='Restrição de leadtime do pedido do cliente i')

def conservacao_fluxo_rule(model, i, k):
    return sum(model.x[k, i, j] for j in model.Nlinha) == sum(model.x[k, j, i] for j in model.Nlinha)
model.conservacao_fluxo = pyo.Constraint(model.Nlinha, model.K, rule=conservacao_fluxo_rule, doc='Restrição de conservação de fluxo')

def tempo_de_servico_rule(model, k, p, i, j):
    return model.t[k, i] + model.s[p] + (model.d[i, j]/model.v) + (model.u[k, i] * model.z[k, i] * (model.r[i]/model.c[k])) <= model.t[k, j]
model.tempo_de_servico = pyo.Constraint(model.K, model.P, model.Nlinha, model.Nlinha, rule=tempo_de_servico_rule, doc='Restrição de tempo de serviço')

def partida_deposito_rule(model, k, zero):
    return sum(model.x[k, zero, j] for j in model.Nlinha) == 1
model.partida_deposito = pyo.Constraint(model.K, model.zero, rule=partida_deposito_rule, doc='Restrição de partida do depósito')

def retorno_deposito_rule(model, k, zero):
    return sum(model.x[k, i, zero] for i in model.Nlinha) == 1
model.retorno_deposito = pyo.Constraint(model.K, model.zero, rule=retorno_deposito_rule, doc='Restrição de retorno ao depósito')

In [ ]:
model.y.display()

In [ ]:
results = opt.solve(model)
results.write()

In [ ]:
for p in model.p:
    for k in model.k:
        for l in model.l:
            if model.QtLM[p,k,l].value > 0:
                print(f'Product {p} from DC {k} to Customer {l}: {model.QtLM[p,k,l].value}')

In [ ]:
for m in model.m:
    for i in model.i:
        for j in model.j:
            if model.QtFM[m,i,j].value > 0:
                print(f'Component {m} from Supplier {i} to Machine {j}: {model.QtFM[m,i,j].value}')

In [ ]:
for p in model.p:
    for j in model.j:
        if model.Qf[p,j].value > 0:
            print(f'Product {p} to produce at Machine {j}: {model.Qf[p,j].value}')

In [ ]:
df_fm = pd.DataFrame(columns=['Flow', 'Item', 'Origin', 'Destination', 'Qty'])
for m, i, j in model.QtFM.extract_values():
    df_ = pd.DataFrame({'Flow':['First Mile'],'Item':[m], 'Origin':[i], 'Destination':[j], 'Qty':[model.QtFM[m,i,j].value]})
    df_fm = pd.concat([df_fm, df_], ignore_index=True)
df_fm = df_fm[df_fm['Qty']>0].copy()

df_mm = pd.DataFrame(columns=['Flow', 'Item', 'Origin', 'Destination', 'Qty'])
for p, j, k in model.QtMM.extract_values():
    df_ = pd.DataFrame({'Flow':['Mid Mile'],'Item':[p], 'Origin':[j], 'Destination':[k], 'Qty':[model.QtMM[p,j,k].value]})
    df_mm = pd.concat([df_mm, df_], ignore_index=True)
df_mm = df_mm[df_mm['Qty']>0].copy()

df_lm = pd.DataFrame(columns=['Flow', 'Item', 'Origin', 'Destination', 'Qty'])
for p, k, l in model.QtLM.extract_values():
    df_ = pd.DataFrame({'Flow':['Last Mile'],'Item':[p], 'Origin':[k], 'Destination':[l], 'Qty':[model.QtLM[p,k,l].value]})
    df_lm = pd.concat([df_lm, df_], ignore_index=True)
df_lm = df_lm[df_lm['Qty']>0].copy()

In [ ]:
folium_map_all = folium.Map(location=(-14.235, -51.9253), tiles='OpenStreetMap', zoom_start=5)
color_options = {
    'Dest':               'gray',
    'First Mile':         'blue',
    'Mid Mile':           'green',
    'Last Mile':          'gray',
}

# First Mile
for supplier in rawmat_suppliers.keys():
    folium.Marker(
      location=(rawmat_suppliers[supplier]['Latitude'], rawmat_suppliers[supplier]['Longitude']),
      popup=f"Raw Material Supplier: {supplier}",
      icon=folium.features.CustomIcon(os.path.join(DATAFILES_DIR, f"{rawmat_suppliers[supplier]['Type']}.png"),icon_size=(25,25)),
      opacity= 1 if supplier in df_fm['Origin'].unique() else .5
    ).add_to(folium_map_all)
   
for _, row in df_fm.iterrows():
    folium.PolyLine([(rawmat_suppliers[row['Origin']]['Latitude'], rawmat_suppliers[row['Origin']]['Longitude']),
                    (factories[machines[row['Destination']]['Factory']]['Latitude'], factories[machines[row['Destination']]['Factory']]['Longitude'])],
                    color=color_options[row['Flow']],
                    opacity=.6).add_to(folium_map_all)

# Mid Mile
for factory in factories.keys():
    folium.Marker(
      location=(factories[factory]['Latitude'], factories[factory]['Longitude']),
      popup=f"Factory: {factory}",
      icon=folium.features.CustomIcon(os.path.join(DATAFILES_DIR, 'Factory.png'),icon_size=(25,25)),
      opacity= 1 if len([machine for machine in factories[factory]['Machines'] if machine in df_mm['Origin'].unique()]) > 0 else .5
    ).add_to(folium_map_all)
for _, row in df_mm.iterrows():
    folium.PolyLine([(factories[machines[row['Origin']]['Factory']]['Latitude'], factories[machines[row['Origin']]['Factory']]['Longitude']),
                    (distribution_centers[row['Destination']]['Latitude'], distribution_centers[row['Destination']]['Longitude'])],
                    color=color_options[row['Flow']],
                    opacity=.6).add_to(folium_map_all)
    

# Last Mile
for dc in distribution_centers.keys():
    folium.Marker(
      location=(distribution_centers[dc]['Latitude'], distribution_centers[dc]['Longitude']),
      popup=f"Distribution Center: {dc}",
      icon=folium.features.CustomIcon(os.path.join(DATAFILES_DIR, 'DC.png'),icon_size=(25,25)),
      opacity= 1 if dc in df_lm['Origin'].unique() else .5
    ).add_to(folium_map_all)
for customer in customers.keys():
    folium.CircleMarker(
        location=(customers[customer]['Latitude'], customers[customer]['Longitude']),
        popup=f"Customer: {customer}",
        radius=df_lm.loc[df_lm['Destination']==customer, 'Qty'].sum()/5,
        color=color_options['Dest'],
        fill=True,
        fill_color=color_options['Dest']
    ).add_to(folium_map_all)

for _, row in df_lm.iterrows():
    folium.PolyLine([(distribution_centers[row['Origin']]['Latitude'], distribution_centers[row['Origin']]['Longitude']),
                    (customers[row['Destination']]['Latitude'], customers[row['Destination']]['Longitude'])],
                    color=color_options[row['Flow']],
                    opacity=.6).add_to(folium_map_all)
    
folium_map_all
folium_map_all.save(os.path.join(OUTPUT_DIR, f'Network Preview.html'))

In [ ]:
model.Tt[('DC4', 'C85', 'Product A')]

In [ ]:
df_fm['Sales Value'] = df_fm.apply(lambda x: rawmat_suppliers[x['Origin']]['Components'][x['Item']], axis=1)
df_fm['Inbound'] = df_fm['Destination'].apply(lambda x: machines[x]['Inbound'])
df_fm['Freight Aliquot'] = df_fm.apply(lambda x: taxes[(x['Origin'], x['Destination'], x['Item'])]['Freight Aliquot'], axis=1)
df_fm['Product Aliquot'] = df_fm.apply(lambda x: taxes[(x['Origin'], x['Destination'], x['Item'])]['Product Aliquot'], axis=1)
df_fm['Taxed BC'] = df_fm.apply(lambda x: taxes[(x['Origin'], x['Destination'], x['Item'])]['Taxed BC'], axis=1)
df_fm['BRL/Kg'] = df_fm.apply(lambda x: model.Ct[(x['Origin'], x['Destination'])], axis=1)

df_fm['Inbound Cost'] = df_fm['Inbound'] * df_fm['Qty'] 
df_fm['Freight Cost'] = df_fm['BRL/Kg'] * df_fm['Qty']
df_fm['Freight Tax Cost'] = df_fm['Freight Cost'] * df_fm['Freight Aliquot']
df_fm['Product Tax Cost'] = (df_fm['Sales Value'] * df_fm['Qty']) * (df_fm['Taxed BC'] * df_fm['Product Aliquot'])

df_fm.head(1)

In [ ]:
df_mm['Sales Value'] = df_mm['Item'].apply(lambda x: products[x]['Kg Price'])
df_mm['Production Value'] = df_mm.apply(lambda x: machines[x['Origin']]['products'][x['Item']]['Prod Cost'], axis=1)
df_mm['Inbound'] = df_mm['Destination'].apply(lambda x: distribution_centers[x]['Inbound'])
df_mm['Outbound'] = df_mm['Origin'].apply(lambda x: machines[x]['Outbound'])
df_mm['Freight Aliquot'] = df_mm.apply(lambda x: taxes[(x['Origin'], x['Destination'], x['Item'])]['Freight Aliquot'], axis=1)
df_mm['Product Aliquot'] = df_mm.apply(lambda x: taxes[(x['Origin'], x['Destination'], x['Item'])]['Product Aliquot'], axis=1)
df_mm['Taxed BC'] = df_mm.apply(lambda x: taxes[(x['Origin'], x['Destination'], x['Item'])]['Taxed BC'], axis=1)
df_mm['BRL/Kg'] = df_mm.apply(lambda x: model.Ct[(x['Origin'], x['Destination'])], axis=1)

df_mm['Inbound Cost'] = df_mm['Inbound'] * df_mm['Qty']
df_mm['Outbound Cost'] = df_mm['Outbound'] * df_mm['Qty']
df_mm['Production Cost'] = df_mm['Production Value'] * df_mm['Qty']
df_mm['Freight Cost'] = df_mm['BRL/Kg'] * df_mm['Qty']
df_mm['Freight Tax Cost'] = df_mm['Freight Cost'] * df_mm['Freight Aliquot']
df_mm['Product Tax Cost'] = (df_mm['Sales Value'] * df_mm['Qty']) * (df_mm['Taxed BC'] * df_mm['Product Aliquot'])


df_mm.head(1)

In [ ]:
df_lm['Sales Value'] = df_lm['Item'].apply(lambda x: products[x]['Kg Price'])
df_lm['Outbound'] = df_lm['Origin'].apply(lambda x: distribution_centers[x]['Outbound'])
df_lm['Freight Aliquot'] = df_lm.apply(lambda x: taxes[(x['Origin'], x['Destination'], x['Item'])]['Freight Aliquot'], axis=1)
df_lm['Product Aliquot'] = df_lm.apply(lambda x: taxes[(x['Origin'], x['Destination'], x['Item'])]['Product Aliquot'], axis=1)
df_lm['Taxed BC'] = df_lm.apply(lambda x: taxes[(x['Origin'], x['Destination'], x['Item'])]['Taxed BC'], axis=1)
df_lm['BRL/Kg'] = df_lm.apply(lambda x: model.Ct[(x['Origin'], x['Destination'])], axis=1)

df_lm['Outbound Cost'] = df_lm['Outbound'] * df_lm['Qty']
df_lm['Freight Cost'] = df_lm['BRL/Kg'] * df_lm['Qty']
df_lm['Freight Tax Cost'] = df_lm['Freight Cost'] * df_lm['Freight Aliquot']
df_lm['Product Tax Cost'] = (df_lm['Sales Value'] * df_lm['Qty']) * (df_lm['Taxed BC'] * df_lm['Product Aliquot'])

df_lm.head(1)

In [ ]:
df_network = pd.concat([df_fm, df_mm, df_lm], ignore_index=True)
df_network['Inbound'] = df_network['Inbound'].fillna(0)
df_network['Outbound'] = df_network['Outbound'].fillna(0)
df_network['Inbound Cost'] = df_network['Inbound Cost'].fillna(0)
df_network['Outbound Cost'] = df_network['Outbound Cost'].fillna(0)
df_network['Production Value'] = df_network['Production Value'].fillna(0)
df_network['Production Cost'] = df_network['Production Cost'].fillna(0)
df_network['Freight Cost'] = df_network['Freight Cost'].fillna(0)
df_network['Freight Tax Cost'] = df_network['Freight Tax Cost'].fillna(0)
df_network['Product Tax Cost'] = df_network['Product Tax Cost'].fillna(0)
df_network['Total Cost'] = df_network['Inbound Cost'] + df_network['Outbound Cost'] + df_network['Production Cost'] + df_network['Freight Cost'] + df_network['Freight Tax Cost'] + df_network['Product Tax Cost']
df_network.to_excel(os.path.join(OUTPUT_DIR, 'Network Costs.xlsx'), index=False)

In [ ]:
#with pd.ExcelWriter(os.path.join(STAGING_DIR, 'Network Costs.xlsx')) as writer:
#    df_fm.to_excel(writer, sheet_name='First Mile', header=True, index=False)
#    df_mm.to_excel(writer, sheet_name='Mid Mile', header=True, index=False)
#    df_lm.to_excel(writer, sheet_name='Last Mile', header=True, index=False)
#    writer.save()